# 注意力机制

In [1]:
import torch

# 创建一个简单的例子
def simple_attention_example():
    # 创建查询和键向量
    query = torch.tensor([[2.0, 3.0]])  # 1x2
    key = torch.tensor([[1.0, 2.0],
                       [0.5, 1.0]])      # 2x2
    
    # 计算注意力分数
    scores = torch.matmul(query, key.T)  # 1x2
    print("原始分数:", scores)
    
    # 缩放
    d_k = query.size(-1)  # = 2
    scaled_scores = scores / torch.sqrt(torch.tensor(d_k).float())
    print("缩放后的分数:", scaled_scores)
    
    # 应用softmax
    attention_weights = torch.nn.functional.softmax(scaled_scores, dim=-1)
    print("注意力权重:", attention_weights)
    
    return attention_weights

# 运行示例
attention_weights = simple_attention_example()

原始分数: tensor([[8., 4.]])
缩放后的分数: tensor([[5.6569, 2.8284]])
注意力权重: tensor([[0.9442, 0.0558]])
